# LLM-Based Ticket Reply Evaluator
**Rauda AI — Take-Home Assignment**

Evalúa respuestas de soporte al cliente usando Llama 3.3 70B vía Groq API.
Para cada par (ticket, reply) el modelo devuelve:
- `content_score` + `content_explanation`
- `format_score` + `format_explanation`

**Escala:** 1 (muy malo) → 5 (excelente)

## 1. Instalación de dependencias

In [1]:
# Las dependencias están en requirements.txt
# Consulta el README para instrucciones de instalación con venv

## 2. Imports y configuración

In [2]:
import os
import json
import logging
import pandas as pd

from groq import Groq
from dotenv import load_dotenv
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type,
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)
logger = logging.getLogger(__name__)

# Cargamos la API key desde .env para no escribirla directamente en el código.
# Así evitamos exponer credenciales si el proyecto se sube a GitHub.
load_dotenv(override=False)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise EnvironmentError(
        "No se encontró GROQ_API_KEY.\n"
        "Crea un archivo .env con: GROQ_API_KEY=gsk_..."
    )

# Configuración centralizada: si hay que cambiar el modelo o los límites,
# se toca aquí y no hay que buscar por todo el código.
MODEL       = "llama-3.3-70b-versatile"
TEMPERATURE = 0.1
MAX_RETRIES = 4
INPUT_FILE  = "../data/tickets.csv"
OUTPUT_FILE = "../data/tickets_evaluated.csv"

client = Groq(api_key=GROQ_API_KEY)
logger.info(f"✅ Cliente Groq inicializado. Modelo: {MODEL}")

2026-02-21 15:06:03,166 [INFO] ✅ Cliente Groq inicializado. Modelo: llama-3.3-70b-versatile


## 3. Prompt Engineering

El **System Prompt** define el rol del modelo y el formato de salida esperado.
El **User Prompt** aporta los datos concretos de cada fila.

Forzamos la respuesta en JSON para poder parsearla de forma fiable sin depender
de que el modelo use siempre el mismo formato de texto libre.

In [3]:
SYSTEM_PROMPT = """
You are an expert quality assurance analyst for a customer support team.
Your task is to evaluate AI-generated replies to customer support tickets.

You will receive a JSON object with two fields:
- "ticket": the original customer message
- "reply": the AI-generated response to evaluate

Evaluate the reply on TWO dimensions using a scale from 1 to 5:

CONTENT (relevance, correctness, completeness):
  5 - Fully addresses all aspects of the ticket; accurate and complete
  4 - Addresses the main issue; minor gaps or imprecisions
  3 - Partially addresses the ticket; some relevant information missing
  2 - Barely addresses the ticket; mostly off-topic or incorrect
  1 - Does not address the ticket at all; irrelevant or harmful

FORMAT (clarity, structure, grammar/spelling):
  5 - Perfectly clear, well-structured, error-free, professional tone
  4 - Clear and professional with minor formatting or grammar issues
  3 - Understandable but with noticeable clarity or grammar problems
  2 - Difficult to read; poor structure or significant grammar errors
  1 - Incomprehensible; severely malformatted or full of errors

CRITICAL INSTRUCTIONS:
- Respond with ONLY a valid JSON object. Nothing else.
- Do NOT use markdown code blocks.
- The JSON must contain EXACTLY these four fields:

{
  "content_score": <integer between 1 and 5>,
  "content_explanation": "<one or two sentences>",
  "format_score": <integer between 1 and 5>,
  "format_explanation": "<one or two sentences>"
}
"""


def build_user_prompt(ticket: str, reply: str) -> str:
    """
    Serializa ticket y reply como JSON para enviarlos al modelo.
    Usar json.dumps() en lugar de f-strings garantiza el escape correcto
    de comillas, saltos de línea y caracteres especiales.
    """
    return json.dumps({"ticket": ticket, "reply": reply}, ensure_ascii=False)


logger.info("System prompt y función de user prompt definidos.")

2026-02-21 15:06:03,195 [INFO] System prompt y función de user prompt definidos.


## 4. Llamada a la API con reintentos automáticos

Usamos `tenacity` para reintentar la llamada si la API falla por rate limit o
por un error temporal. El backoff exponencial hace que espere cada vez más
entre reintentos, evitando saturar el servicio.

In [4]:
@retry(
    retry=retry_if_exception_type(Exception),
    stop=stop_after_attempt(MAX_RETRIES),
    wait=wait_exponential(multiplier=1, min=2, max=60),
    before_sleep=lambda retry_state: logger.warning(
        f"  ⚠️ Reintento {retry_state.attempt_number}/{MAX_RETRIES} "
        f"tras error: {retry_state.outcome.exception()}"
    ),
)
def call_llm_api(ticket: str, reply: str) -> dict:
    """
    Llama al LLM y devuelve un dict con los 4 campos de evaluación.

    Args:
        ticket: Mensaje original del cliente.
        reply:  Respuesta del sistema de IA a evaluar.

    Returns:
        Dict con content_score, content_explanation, format_score, format_explanation.

    Raises:
        ValueError: Si la respuesta no cumple el schema esperado.
    """
    response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": build_user_prompt(ticket, reply)},
        ],
    )

    result = json.loads(response.choices[0].message.content.strip())

    # Verificamos que el modelo devolvió todos los campos que necesitamos
    required_fields = {
        "content_score", "content_explanation",
        "format_score",  "format_explanation",
    }
    missing = required_fields - result.keys()
    if missing:
        raise ValueError(f"Campos faltantes en la respuesta: {missing}")

    # Verificamos que los scores son enteros en el rango correcto
    for field in ("content_score", "format_score"):
        score = result[field]
        if not isinstance(score, int) or not (1 <= score <= 5):
            raise ValueError(f"Score inválido en '{field}': {score!r}")

    return result


logger.info("Función de llamada a API definida con retry automático.")

2026-02-21 15:06:03,213 [INFO] Función de llamada a API definida con retry automático.


## 5. Evaluación de todos los tickets

In [5]:
def evaluate_tickets(df: pd.DataFrame) -> pd.DataFrame:
    """
    Itera sobre el DataFrame y evalúa cada par (ticket, reply).
    Si una fila falla, registra el error y continúa con las siguientes
    en lugar de detener todo el proceso.

    Args:
        df: DataFrame con columnas 'ticket' y 'reply'.

    Returns:
        DataFrame original con las 4 columnas de evaluación añadidas.
    """
    results = []
    total = len(df)

    for idx, row in df.iterrows():
        logger.info(f"Evaluando fila {idx + 1}/{total}...")

        ticket = str(row.get("ticket", "")).strip()
        reply  = str(row.get("reply",  "")).strip()

        if not ticket or not reply:
            logger.warning(f"Fila {idx}: datos vacíos, se omite.")
            results.append({
                "content_score": None, "content_explanation": "Missing data",
                "format_score":  None, "format_explanation":  "Missing data",
            })
            continue

        try:
            evaluation = call_llm_api(ticket, reply)
            results.append(evaluation)
            logger.info(
                f"  ✓ Content: {evaluation['content_score']}/5 | "
                f"Format: {evaluation['format_score']}/5"
            )
        except Exception as e:
            logger.error(f"  ✗ Error permanente en fila {idx}: {e}")
            results.append({
                "content_score": None, "content_explanation": f"Error: {e}",
                "format_score":  None, "format_explanation":  f"Error: {e}",
            })

    return pd.concat([df.reset_index(drop=True), pd.DataFrame(results)], axis=1)


logger.info("Función de evaluación definida.")

2026-02-21 15:06:03,239 [INFO] Función de evaluación definida.


## 6. Lectura y validación del CSV de entrada

In [6]:
def load_and_validate_csv(filepath: str) -> pd.DataFrame:
    """
    Lee el CSV y comprueba que tiene las columnas necesarias antes de
    hacer ninguna llamada a la API.

    Args:
        filepath: Ruta al archivo CSV.

    Returns:
        DataFrame validado.

    Raises:
        FileNotFoundError: Si el archivo no existe.
        ValueError: Si faltan columnas obligatorias.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No se encontró: {filepath}")

    df = pd.read_csv(filepath)
    logger.info(f"CSV cargado: {len(df)} filas | columnas: {list(df.columns)}")

    missing_cols = {"ticket", "reply"} - set(df.columns)
    if missing_cols:
        raise ValueError(f"Columnas faltantes en el CSV: {missing_cols}")

    empty_rows = df[["ticket", "reply"]].isnull().any(axis=1).sum()
    if empty_rows > 0:
        logger.warning(f"{empty_rows} filas con valores nulos — se omitirán.")

    return df


df_input = load_and_validate_csv(INPUT_FILE)
df_input.head()

2026-02-21 15:06:03,250 [INFO] CSV cargado: 5 filas | columnas: ['ticket', 'reply']


,ticket,reply
0,"Hi, I'd like to check the shipping status of m...","Sure, you can check your shipping status on ou..."
1,The product I received is defective. I'd like ...,We're sorry to hear that. Could you please pro...
2,I have a question about how to set up my new d...,"Hello, thanks for contacting us. Please go to ..."
3,"Hello, my account was charged twice. Could you...",We've identified the issue and have requested ...
4,I can't log into my account. It says 'invalid ...,Please reset your password using the 'Forgot P...


## 7. Evaluación y escritura del resultado

In [7]:
df_evaluated = evaluate_tickets(df_input)

output_columns = [
    "ticket", "reply",
    "content_score", "content_explanation",
    "format_score",  "format_explanation",
]
df_output = df_evaluated[output_columns]

df_output.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
logger.info(f"✅ Evaluación completada. Resultado guardado en: {OUTPUT_FILE}")

df_output

2026-02-21 15:06:03,312 [INFO] Evaluando fila 1/5...
2026-02-21 15:06:03,984 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-21 15:06:03,997 [INFO]   ✓ Content: 4/5 | Format: 5/5
2026-02-21 15:06:03,997 [INFO] Evaluando fila 2/5...
2026-02-21 15:06:04,499 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-21 15:06:04,501 [INFO]   ✓ Content: 4/5 | Format: 5/5
2026-02-21 15:06:04,502 [INFO] Evaluando fila 3/5...
2026-02-21 15:06:05,038 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-21 15:06:05,040 [INFO]   ✓ Content: 4/5 | Format: 5/5
2026-02-21 15:06:05,041 [INFO] Evaluando fila 4/5...
2026-02-21 15:06:05,329 [INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-21 15:06:05,329 [INFO]   ✓ Content: 5/5 | Format: 5/5
2026-02-21 15:06:05,329 [INFO] Evaluando fila 5/5...
2026-02-21 15:06:05,704

,ticket,reply,content_score,content_explanation,format_score,format_explanation
0,"Hi, I'd like to check the shipping status of m...","Sure, you can check your shipping status on ou...",4,The reply addresses the main issue by providin...,5,"The reply is clear, well-structured, and free ..."
1,The product I received is defective. I'd like ...,We're sorry to hear that. Could you please pro...,4,The reply addresses the main issue by acknowle...,5,"The reply is clear, well-structured, and error..."
2,I have a question about how to set up my new d...,"Hello, thanks for contacting us. Please go to ...",4,The reply addresses the main issue by providin...,5,"The reply is clear, well-structured, and free ..."
3,"Hello, my account was charged twice. Could you...",We've identified the issue and have requested ...,5,The reply fully addresses the customer's issue...,5,"The response is clear, concise, and well-struc..."
4,I can't log into my account. It says 'invalid ...,Please reset your password using the 'Forgot P...,4,The reply addresses the main issue by providin...,5,"The reply is clear, well-structured, and free ..."


## 8. Resumen de resultados

In [8]:
print("=" * 55)
print("          RESUMEN DE EVALUACIÓN")
print("=" * 55)
print(f"Total de tickets procesados : {len(df_output)}")
print(f"Content Score — Media: {df_output['content_score'].mean():.2f} "
      f"| Min: {df_output['content_score'].min()} "
      f"| Max: {df_output['content_score'].max()}")
print(f"Format Score  — Media: {df_output['format_score'].mean():.2f} "
      f"| Min: {df_output['format_score'].min()} "
      f"| Max: {df_output['format_score'].max()}")
print(f"Filas con error             : {df_output['content_score'].isnull().sum()}")
print("=" * 55)

          RESUMEN DE EVALUACIÓN
Total de tickets procesados : 5
Content Score — Media: 4.20 | Min: 4 | Max: 5
Format Score  — Media: 5.00 | Min: 5 | Max: 5
Filas con error             : 0


---
## 🎯 Escalabilidad: qué haría con 1 millón de tickets

### 1. Procesamiento concurrente con asyncio
La versión actual procesa un ticket cada vez. Con `asyncio` y llamadas concurrentes
controladas por un semáforo (`asyncio.Semaphore(50)`), el tiempo se reduciría
de días a horas.

### 2. Arquitectura de cola con AWS SQS + Lambda
Para escala real: **S3** recibe el CSV → **Lambda** publica cada fila en **SQS** →
múltiples **Lambda workers** procesan en paralelo → resultados en **DynamoDB**.
Ventajas: escalado automático, tolerancia a fallos con Dead Letter Queue,
coste pay-per-execution.

### 3. Control de costes y observabilidad
- Caché semántica (Redis) para tickets similares ya evaluados
- Modelo más barato para el grueso, modelo premium solo para scores 2-3
- Alertas de presupuesto y logging de tokens consumidos por llamada